In [2]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import os
import tqdm
import pandas as pd
import re

In [2]:
root_url = 'https://collections.mfa.org'
page_num = 1
browse_url = 'https://collections.mfa.org/objects/images?page={}'.format(page_num)

In [3]:
records = []

In [4]:
def download_paint_grids(grid_soup):
    paint_url_raw = grid_soup.find(class_='primaryMedia').find('a')['href']
    paint_url_rel = re.match(r'/objects/(\d+)', paint_url_raw).group(0)
    paint_url = urllib.parse.urljoin(root_url, paint_url_rel)
    #
    try:
        with urllib.request.urlopen(paint_url) as response:
            paint_html = response.read()
    except Exception as e:
        print(e)
        return
    paint_soup = BeautifulSoup(paint_html)
    # Get Image Object ID
    try:
        img_rel = paint_soup.find(class_="download-image").img.get('src').split(';')[0]
    except Exception as e:
        object_id = None
    else:
        img_abs = urllib.parse.urljoin(paint_url, img_rel)
        object_id = img_abs.split('/')[-2]
    # Get Painting Information
    try:
        details = str(paint_soup.find(class_='item-details-inner'))
    except Exception as e:
        print(e)
        return
    return paint_url, object_id, details

In [5]:
def download_browse_page(page_num):
    browse_url = 'https://collections.mfa.org/objects/images?page={}'.format(page_num)
    with urllib.request.urlopen(browse_url) as response:
        browse_html = response.read()
    browse_soup = BeautifulSoup(browse_html, 'html.parser')
    paint_grids = browse_soup.find_all(class_='grid-item-inner')
    for grid_soup in paint_grids:
        record = download_paint_grids(grid_soup)
        if record is None:
            continue
        else:
            records.append(record)
    return browse_soup

# Pykka

In [4]:
import pykka

ModuleNotFoundError: No module named 'pykka'

In [6]:
from queue import Queue
import threading

In [7]:
num_worker_threads = 10

In [8]:
def worker(q):
    while True:
        print(q.qsize())
        page_num = q.get()
        if page_num is None:
            break
        download_browse_page(page_num)
        q.task_done()

In [ ]:
q = Queue()
threads = []
for _ in range(num_worker_threads):
    t = threading.Thread(target=worker, args=(q,))
    t.start()
    threads.append(t)
for page_num in range(1, 36117):
    q.put(page_num)
q.join()
for _ in range(num_worker_threads):
    q.put(None)
for t in threads:
    t.join()

0
00
0
0
00
00
0



36106
36105
36104
36103
36102
36101
36100
36099
36098
36097
36096
36095
36094
36093
36092
36091
36090
36089
36088
36087
36086
36085
36084
36083
36082
36081
36080
36079
36078
36077
